In [93]:
## Import python library
import pandas as pd
import nltk

## Read file
file_name = 'OPPO A31.csv'
## Read file using pandas
df = pd.read_csv(file_name)

In [94]:
# imports
import re
# function to remove special characters
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

In [95]:
import re
l = []
asd_train = []
pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
for i in range(df.shape[0]):
    
    g = str(df.iloc[i]['body'])
    g = g.replace('[^\w\s]','')
    g = remove_special_characters(g)
    g = g.replace("/"," ")
    g = g.replace("."," ").replace(","," ").replace("\'","").replace("\"","").replace(":"," ")
    g = " ".join(g.split())
    k = ""
    for j in g:
        if(j == " "):
            k = k + j
        elif(ord(j)>=ord('A') and ord(j)<=ord('Z')):
            k = k + j
        elif(ord(j)>=ord('a') and ord(j)<=ord('z')):
            k = k + j
                
    g1 = df.iloc[i]['rating']
    if(g1>3):
        asd_train.append(1)
    else:
        asd_train.append(0)
    l.append(k)

In [96]:
df['body1'] = l

In [97]:
df['rating'] = asd_train

In [98]:
df.head()

,product,title,rating,body,body1
0,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",earphones levu. and Mobile software working s...,0,I am not happy buy the mobile in amazon.Whats ...,I am not happy buy the mobile in amazon Whats ...
1,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Oppo a31,1,Best Product Thnx for Amazon.in,Best Product Thnx for Amazon in
2,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Awesome,1,Very nice phone.Eye protection also there.Awes...,Very nice phone Eye protection also there Awes...
3,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Complaint,0,Hi i received this item on 16 july.after 10 da...,Hi i received this item on july after days p...
4,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Waste of money. Please don't buy it,0,Please don't buy this mobile. Waste of money. ...,Please dont buy this mobile Waste of money I b...


In [99]:
df.rating.value_counts()

1    3286
0    1589
Name: rating, dtype: int64

In [100]:
# load nltk's English stopwords as variable called 'stop' and don't find synonym of those words.
stop = nltk.corpus.stopwords.words('english')

In [101]:
## Tokenizing sentence into token for finding synonym.
def make_tokenizer(texts):
    from keras.preprocessing.text import Tokenizer
    t = Tokenizer()
    t.fit_on_texts(texts)
    return t

tokenizer = make_tokenizer(df['body1'])    ## Message is column name

X = tokenizer.texts_to_sequences(df['body1'])

from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, 70)

In [102]:
## Dictionary of word index
index_word = {}
for word in tokenizer.word_index.keys():
    index_word[tokenizer.word_index[word]] = word

In [103]:
## word list
words = [value for key, value in index_word.items()]

In [104]:
## Function to find synonym of words 
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser'])
def check_lemma(t,w) :
    r = [d for d in t if (nlp(d.text)[0].lemma_ != nlp(w.text)[0].lemma_)]
    return r

def get_word_synonym(word):
    filtered_words = [w for w in word.vocab if (not w.lower_ in stop) and w.is_lower == word.is_lower and w.prob >= -15] ## (not w.lower_ in stop) and
    similarity = sorted(filtered_words, key=lambda w: word.similarity(w), reverse=True)
    filtered_similarity = check_lemma(similarity[:30], word)
    return filtered_similarity[:3]

c:\python39\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  raise FileNotFoundError(


In [105]:
## Synonym dictionary
synonym_dict = {}

for word in words:
    synonym_dict.update({word : tuple([w.lower_ for w in get_word_synonym(nlp.vocab[word])])})
        #print(word, " : ", [w.lower_ for w in get_word_synonym(nlp.vocab[word])])

In [106]:
## Only consider filtered synonym
import collections
value_occurrences = collections.Counter(synonym_dict.values())

filtered_synonym = {key: value for key, value in synonym_dict.items() if value_occurrences[value] == 1}

In [107]:
## Function for augmenting data by replacing words with synonym using spaCy
## This might not be best best method compared to data augmentation using language translation

import re
import random
sr = random.SystemRandom()
split_pattern = re.compile(r'\s+')
def data_augmentation(message, aug_range=1) :
    augmented_messages = []
    for j in range(0,aug_range) :
        new_message = ""
        for i in filter(None, split_pattern.split(message)) :
            new_message = new_message + " " + sr.choice(filtered_synonym.get(i,[i]))
        augmented_messages.append(new_message)
    return augmented_messages

In [108]:
## Dictionary for intent count
## Intent is column name
intent_count = df.rating.value_counts().to_dict()

In [109]:
## Get max intent count to match other minority classes through data augmentation
import operator
max_intent_count = max(intent_count.items(), key=operator.itemgetter(1))[1]

In [110]:
## Loop to interate all messages
import numpy as np
import math
import tqdm
newdf = pd.DataFrame()
for intent, count in intent_count.items() :
    count_diff = max_intent_count - count    ## Difference to fill
    multiplication_count = math.ceil((count_diff)/count)  ## Multiplying a minority classes for multiplication_count times
    if (multiplication_count) :
        old_message_df = pd.DataFrame()
        new_message_df = pd.DataFrame()
        for message in tqdm.tqdm(df[df["rating"] == intent]["body1"]) :
            ## Extracting existing minority class batch
            dummy1 = pd.DataFrame([message], columns=['body1'])
            dummy1["rating"] = intent
            old_message_df = old_message_df.append(dummy1)
            
            ## Creating new augmented batch from existing minority class
            new_messages = data_augmentation(message, multiplication_count)
            dummy2 = pd.DataFrame(new_messages, columns=['body1'])
            dummy2["rating"] = intent
            new_message_df = new_message_df.append(dummy2)
        
        ## Select random data points from augmented data
        new_message_df=new_message_df.take(np.random.permutation(len(new_message_df))[:count_diff])
        
        ## Merge existing and augmented data points
        newdf = newdf.append([old_message_df,new_message_df])
    else :
        newdf = newdf.append(df[df["rating"] == intent])

100%|█████████████████████████████████████████████████████████████████████████████| 1589/1589 [00:02<00:00, 531.62it/s]


In [111]:
newdf.rating.value_counts()

1    3286
0    3286
Name: rating, dtype: int64

In [112]:
newdf.head()

,product,title,rating,body,body1
1,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Oppo a31,1,Best Product Thnx for Amazon.in,Best Product Thnx for Amazon in
2,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Awesome,1,Very nice phone.Eye protection also there.Awes...,Very nice phone Eye protection also there Awes...
5,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Osm,1,Nice,Nice
7,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Worthy n best product,1,It's worth for 9.5k. after exchange offer.ligh...,Its worth for k after exchange offer light we...
9,"OPPO A31 (Fantasy White, 6GB RAM, 128GB Storag...",Very Good product,1,Its a really very amazing and a nicely working...,Its a really very amazing and a nicely working...


In [113]:
newdf.tail()

,product,title,rating,body,body1
0,NaN,NaN,0,NaN,Totally waste
1,NaN,NaN,0,NaN,
0,NaN,NaN,0,NaN,The phone is totally rubbish the battery does...
0,NaN,NaN,0,NaN,Waste of cost Zero quality this sellar is a f...
0,NaN,NaN,0,NaN,Battery slow


In [115]:
newdf.to_csv("OPPO A31 Balanced.csv")